In [1]:
import pickle
import cv2
import os
import numpy as np
import time

In [2]:
import tensorflow as tf
model = tf.keras.models.load_model("./_model")

In [3]:
def classify(img): # returns 0 for closed, 1 for open
    img = cv2.resize(img,(224,224))
    img = np.array(img)
    img = img / 255.0
    img = np.expand_dims(img, axis=-1)
    img = np.repeat(img, 3, axis=-1)
    img = np.expand_dims(img, axis=0)
    y_pred = model.predict(img,verbose = 0)  
    y_pred = np.round(y_pred)
    y_pred = int(y_pred[0][0])
    return y_pred

In [4]:
from imutils import face_utils
import dlib
import cv2
from IPython.display import clear_output, display
 
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)


In [ ]:

cap = cv2.VideoCapture(0)
v = 0
ctr = 0
begin = time.time()
prev_label = -1
org = (50, 50)
fontScale = 0.7
color = (255, 0, 0)
thickness = 2
font = 2
while True:
    _, image = cap.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        
        ratio = 1.3
        xmi = shape.part(0).x
        xma = shape.part(16).x
        xcent = (xma+xmi)/2
        l = ((xma-xmi)*ratio)/2
        ycent = shape.part(19).y+shape.part(24).y+shape.part(8).y
        ycent /= 3
        xmin = int(xcent - l)
        xmax = int(xcent+l)
        ymin = int(ycent - l)
        ymax = int(ycent+l)
        im1 = gray[ymin:ymax,xmin:xmax]
        label = classify(im1)
        
        if(prev_label == 1 and label == 0):
            ctr+=1
        prev_label = label
        curr = round((ctr*60)/(time.time() - begin),2)
        image = cv2.rectangle(image,(xmin,ymin), (xmax,ymax), (255,0,255), 2)

        image = cv2.putText(image, 'Eyes:'+str(label)+'      Blinks: '+str(ctr)+'      Time: '+str(curr)+' blinks/min', org, font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow("Feed",image)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 13:
        break

cv2.destroyAllWindows()
cap.release()

In [ ]:

# cap = cv2.VideoCapture(0)
# v = 0
# while True:
#     _, image = cap.read()
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     rects = detector(gray, 0)
#     for (i, rect) in enumerate(rects):
#         shape = predictor(gray, rect)
#         ratio = 1.6
#         xmi = shape.part(0).x
#         xma = shape.part(16).x
#         xcent = (xma+xmi)/2
#         l = ((xma-xmi)*ratio)/2
#         ycent = shape.part(19).y+shape.part(24).y+shape.part(8).y
#         ycent /= 3
#         xmin = xcent - l
#         xmax = xcent+l
#         ymin = ycent - l
#         ymax = ycent+l
#         xmin = int(xmin)
#         xmax = int(xmax)
#         ymin = int(ymin)
#         ymax = int(ymax)
#         im1 = gray[ymin:ymax,xmin:xmax]
#         label = classify(im1)
#         gray[ymin:ymax,xmin:xmax]-=40
        
#         cv2.imshow("Feed",gray)
#         print("\r"+str(label), end="")
# #         for i in range(36,42):
# #             cv2.circle(image, (shape.part(i).x, shape.part(i).y), 2, (0, 255, 0), -1)
    
#     # Show the image
# #     cv2.imshow("Output", image)
    
#     k = cv2.waitKey(5) & 0xFF
#     if k == 13:
#         break

# cv2.destroyAllWindows()
# cap.release()